In [1]:
class DictaQueryStatement:
    def __init__(self, parent, property_name, 
                 func_name, value):
        print('DictaQueryStatement: {} {} {}'.format(property_name, func_name, value))
        self.parent = parent
        self.property_name = property_name
        self.func_name = func_name
        self.value = value
        
    def __call__(self):
        dict_val = self.parent[self.property_name]
        func = getattr(dict_val, self.func_name)
        result = func(self.value)
        print("Is {} {} {}? {}".format(dict_val, self.func_name, self.value, result))
        return result

class DictaQuery:
    def __init__(self, parent, statements):
        self.parent = parent
        self.statements = statements
        print('DictaQuery: {} statements'.format(len(self.statements)))
        
    def filter(self, statement):
        return DictaQuery(
            self.parent, 
            self.statements + [statement])
    
    def run(self):
        answer = Dictabase()
        
        for statement in self.statements:
            if statement():
                prop = statement.property_name
                answer[prop] = statement.parent[prop]
                    
        return answer
    
class DictaProperty:
    
    def __init__(self, parent, property_name):
        print('DictaProperty {}'.format(property_name))
        self.parent = parent
        self.property_name = property_name
    
    def __eq__(self, value):
        return DictaQueryStatement(
            self.parent, 
            self.property_name, 
            '__eq__', 
            value)
    
    def __ge__(self, value):
        return DictaQueryStatement(
            self.parent, 
            self.property_name, 
            '__ge__', 
            value)
    
    def __ne__(self, value):
        return DictaQueryStatement(
            self.parent, 
            self.property_name, 
            '__ne__', 
            value)

    def __lt__(self, value):
        return DictaQueryStatement(
            self.parent, 
            self.property_name, 
            '__lt__', 
            value)

    def __le__(self, value):
        return DictQueryNode(
            self.parent, 
            self.property_name, 
            '__le__', 
            value)

    def __gt__(self, value):
        return DictaQueryStatement(
            self.parent, 
            self.property_name, 
            '__gt__', 
            value)


    
class Dictabase(dict):
    
    def __getattr__(self, property_name):
        return DictaProperty(
            parent=self, 
            property_name=property_name)
    
    def query(self, statement=None):
        statements = [statement] if statement else []
        return DictaQuery(self, statements)
        
    def __enter__(self):
        return DictaQuery(self, [])
    
    def __exit__(self, *args, **kwargs):
        pass

In [2]:
puppy = Dictabase(
    Don='Sheu', 
    Alan='Vezzinna', 
    Nick='Denny')
print()
query = puppy.query()
print()
query = query.filter(puppy.Nick >= 'Hunter-Walker')
print()
query = query.filter(puppy.Don >= 'Duck')
print()
query = query.filter(puppy.Alan >= None)
print()
puppy['Alan'] = 'Vezina'
print()
results = query.run()
print()
print(results)


with puppy as query:
    query = query.filter(puppy.Nick >= 'D')
    print(query.run())


DictaQuery: 0 statements

DictaProperty Nick
DictaQueryStatement: Nick __ge__ Hunter-Walker
DictaQuery: 1 statements

DictaProperty Don
DictaQueryStatement: Don __ge__ Duck
DictaQuery: 2 statements

DictaProperty Alan
DictaQueryStatement: Alan __ge__ None
DictaQuery: 3 statements


Is Denny __ge__ Hunter-Walker? False
Is Sheu __ge__ Duck? True
Is Vezina __ge__ None? NotImplemented

{'Don': 'Sheu', 'Alan': 'Vezina'}
DictaQuery: 0 statements
DictaProperty Nick
DictaQueryStatement: Nick __ge__ D
DictaQuery: 1 statements
Is Denny __ge__ D? True
{'Nick': 'Denny'}
